## 导入工具包

In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Circle, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster

## 坐标转换

In [15]:
# -*- coding: utf-8 -*-
import json
import urllib
import math

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方


'''
输入（经度，维度）
'''
def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]
def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]
def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)
def gcj02_to_bd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]
def wgs84_to_gcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]
def wgs84_to_bd09(lon, lat):
    lon, lat = wgs84_to_gcj02(lon, lat)
    return gcj02_to_bd09(lon, lat)

def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def bd_to_wgs(row):
	nlng, nlat = bd09_to_wgs84(row.lng, row.lat)
	row.lng = nlng
	row.lat = nlat
	return row


## 数据准备

In [38]:
huoguo = pd.read_csv('./data/huoguo.csv')
huoguo = huoguo.apply(bd_to_wgs, axis='columns')

huoguo_geo = gpd.GeoDataFrame(
    huoguo, geometry=gpd.points_from_xy(huoguo.lng, huoguo.lat))
huoguo_geo.crs = {'init': 'epsg:4326'}
huoguo_geo.head()

/Users/hawkins/opt/anaconda3/envs/analysis/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,name,lng,lat,address,geometry
0,蜀大侠火锅(春熙店),104.073487,30.656484,四川省成都市锦江区上东大街6号春南商场2层,POINT (104.07349 30.65648)
1,锦城印象(高新店),104.068731,30.592815,成都市武侯区天顺路88号,POINT (104.06873 30.59281)
2,小龙坎火锅(春熙太古里店),104.077745,30.653506,四川省成都市锦江区下东大街36号郁金香广场2楼,POINT (104.07774 30.65351)
3,园里火锅(天府三街店),104.063460,30.550215,四川省成都市武侯区天府三街大源国际中心B1栋3层,POINT (104.06346 30.55022)
4,巴蜀大宅门火锅(川音店),104.081647,30.641902,四川省成都市武侯区老马路14号,POINT (104.08165 30.64190)


In [39]:
naicha = pd.read_csv('./data/naicha.csv')
naicha = naicha.apply(bd_to_wgs, axis='columns')

naicha_geo = gpd.GeoDataFrame(naicha, geometry=gpd.points_from_xy(naicha.lng, naicha.lat))
naicha_geo.crs = {'init': 'epsg:4326'}
naicha.head()

/Users/hawkins/opt/anaconda3/envs/analysis/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,name,lng,lat,address,geometry
0,书亦烧仙草(熊猫大道店),104.113218,30.724609,四川省成都市成华区将军碑东二路88号3栋107号,POINT (104.11322 30.72461)
1,茶百道(财富又一城店),104.090405,30.680662,四川省成都市成华区前锋路1号附17号,POINT (104.09040 30.68066)
2,书亦烧仙草(新华公园店),104.104336,30.659915,成都市成华区双林北支路105号附14号,POINT (104.10434 30.65991)
3,书亦烧仙草(金堂华地财富广场店),104.396859,30.863022,四川省成都市金堂县金园街120号,POINT (104.39686 30.86302)
4,蜜雪冰城(洛带古镇店),104.324242,30.640338,四川省成都市龙泉驿区下街194号,POINT (104.32424 30.64034)


## 基本数据展示
火锅店用热力图展示，奶茶店用标记

In [43]:
# 天府广场,104.072425,30.663503,四川省成都市锦江区人民南路一段86号
ct_lon, ct_lat = bd09_to_wgs84(104.072425, 30.663503)

m_1 = folium.Map(location=[ct_lat, ct_lon], zoom_start=11)

HeatMap(data=huoguo_geo[['lat','lng']], radius=20).add_to(m_1)
mc = MarkerCluster()
for idx, row in naicha_geo.iterrows():
	Marker([row.lat, row.lng], popup=row['name']).add_to(mc)

mc.add_to(m_1)

m_1

## 那些火锅店附件2km没有奶茶店
找到2km内没有奶茶店的火锅店

In [42]:
# 球面坐标系转换为平面坐标系才能计算距离
naicha_geo = naicha_geo.to_crs(crs=3857)
huoguo_geo = huoguo_geo.to_crs(crs=3857)

one_buffer = naicha_geo.geometry.buffer(2000)
union_buffer = one_buffer.geometry.unary_union

huoguo_outside_range = huoguo_geo.loc[~huoguo_geo.geometry.apply(lambda x:union_buffer.contains(x))]
huoguo_outside_range

,name,lng,lat,address,geometry
3,园里火锅(天府三街店),104.063460,30.550215,四川省成都市武侯区天府三街大源国际中心B1栋3层,POINT (11584291.399 3574472.738)
10,玛歌庄园火锅(五龙山店),104.165840,30.764306,四川省成都市新都区叠秀路1676号,POINT (11595688.322 3602177.477)
20,泥巴小院市井火锅(锦华店),104.096227,30.622238,四川省成都市锦江区东光街道锦华路一段163号附一号奔驰4s店旁,POINT (11587938.969 3583786.101)
24,月满大江火锅(科华店),104.074161,30.618022,四川省成都市武侯区科华北路君悦尚都一楼,POINT (11585482.648 3583240.708)
25,龙腾庭院火锅(恒大新城店),103.864788,30.667297,四川省成都市温江区东坡路西五段兴元腾达汽修厂西100米,POINT (11562175.286 3589616.274)
...,...,...,...,...,...
431,郭氏冒菜(黄河路店),104.188008,30.817665,四川省成都市新都区黄河路12-1-1号,POINT (11598155.994 3609092.105)
434,姐妹串串(大源西街店),104.030593,30.558239,四川省成都市武侯区大源西街82号,POINT (11580632.690 3575510.017)
437,三顾冒菜(龙湖北城天街店),104.062081,30.712261,成都市金牛区五块石路1号龙湖北城天街购物中心F1,POINT (11584137.865 3595436.921)
438,刘幺妹砂锅冒菜(东大街店),103.942169,30.985720,四川省成都市彭州市东大街24号4栋附106号,POINT (11570789.320 3630894.802)


画出奶茶店和附件没有奶茶店的火锅店的热力图

In [45]:
# 以天府广场为中心
m_2 = folium.Map(location=[ct_lat, ct_lon],
                 tiles='OpenStreetMap', zoom_start=11)

# 以奶茶店为中心，画两公里的圆
coverage = gpd.GeoDataFrame(geometry=naicha_geo.geometry).buffer(2000)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_2)  # 要画图需要先把坐标转换回来

# 只包含不在两公里内的火锅店的热力图
HeatMap(data=huoguo_outside_range[['lat', 'lng']], radius=20).add_to(m_2)
folium.LatLngPopup().add_to(m_2)

m_2


## 计算客流量最大的奶茶店

In [48]:
# Create a map
m_3 = folium.Map(location=[30.571925, 104.081128],
                 tiles='openstreetmap', zoom_start=11)


def best_naicha(huogou_loc):
	idx_min = naicha_geo.geometry.distance(huogou_loc).idxmin()
	sx = naicha_geo.iloc[idx_min]
	return sx['name']


highest_demand = huoguo_outside_range.geometry.apply(best_naicha).value_counts().idxmax()
# highest_demand = huoguo_outside_range.geometry.apply(
# 	best_naicha).value_counts()
highest_demand

'相遇奶茶屋'

## 最后的选择

In [63]:
# 火车南站附近
lat_1 = 30.6088
long_1 = 104.0652

# 光华大道
lat_2 = 30.6726
long_2 = 104.0000

new_df = pd.DataFrame(
    {'lat': [lat_1, lat_2],
     'lng': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(
    new_df, geometry=gpd.points_from_xy(new_df.lng, new_df.lat))
new_gdf.crs = {'init': 'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=3857)

new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(2000)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = huoguo_outside_range.loc[~huoguo_outside_range["geometry"].apply(
    lambda x: new_my_union.contains(x))]

new_percentage = round(100*len(new_outside_range)/len(huoguo_geo), 2)
old_percentage = round(100*len(huoguo_outside_range)/len(huoguo_geo), 2)
print("当前附件没有奶茶店的比例: {}%, 以前的比例：{}%".format(new_percentage, old_percentage))

m_5 = folium.Map(location=[ct_lat, ct_lon],
                  tiles='OpenStreetMap', zoom_start=11)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_5)
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m_5)
for idx, row in new_gdf.iterrows():
		Marker([row['lat'], row['lng']]).add_to(m_5)
HeatMap(data=new_outside_range[['lat', 'lng']], radius=20).add_to(m_5)
folium.LatLngPopup().add_to(m_5)
m_5


当前附件没有奶茶店的比例: 32.81%, 以前的比例：37.75%


/Users/hawkins/opt/anaconda3/envs/analysis/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
